In [1]:
%env SPARK_VERSION=3.0.0 # TODO PUT YOUR VALUE

env: SPARK_VERSION=3.0.0 # TODO PUT YOUR VALUE


In [2]:
# preparing environment
import sys
import urllib.request
import os

!{sys.executable} -m pip install pydeequ
# findspark sagemaker_pyspark
with urllib.request.urlopen("https://go.microsoft.com/fwlink/?linkid=2122434") as response, open("/home/jovyan/work/mssql-jdbc-7.2.2.tar.gz", 'wb') as out_file:
    data = response.read() # a `bytes` object
    out_file.write(data)
    
os.system('cd /home/jovyan/work/ && tar zxvf mssql-jdbc-7.2.2.tar.gz sqljdbc_7.2/enu/mssql-jdbc-7.2.2.jre11.jar && mv sqljdbc_7.2/enu/mssql-jdbc-7.2.2.jre11.jar .')
# !{sys.executable} tar zxvf /home/jovyan/work/mssql-jdbc-7.2.2.tar.gz sqljdbc_7.2/enu/mssql-jdbc-7.2.2.jre11.jar
# os.system("pip3 install sagemaker_pyspark")
# os.system("/usr/local/spark-3.0.0-bin-hadoop3.2/bin/pyspark --packages ch.cern.sparkmeasure:spark-measure_2.12:0.17")


sqljdbc_7.2/enu/mssql-jdbc-7.2.2.jre11.jar


0

In [3]:
import pydeequ
# import findspark
# findspark.init()

from pyspark import SparkContext, SparkConf, SQLContext

appName = "PySpark SQL Server Example - via JDBC"
master = "local[*]"
conf = SparkConf() \
    .setAppName(appName) \
    .setMaster(master) \
    .set("spark.driver.extraClassPath","/home/jovyan/work/mssql-jdbc-9.4.0.jre11.jar") \
    .set("spark.jars.packages",pydeequ.deequ_maven_coord) \
    .set("spark.jars.excludes",pydeequ.f2j_maven_coord)
# mssql-jdbc-7.2.2.jre11.jar 

sc = SparkContext.getOrCreate(conf=conf)
sqlContext = SQLContext(sc)

spark = sqlContext.sparkSession

# spark = (sqlContext.sparkSession
#     .builder
#     .config("spark.jars.packages", pydeequ.deequ_maven_coord)
#     .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
#     .getOrCreate())

print(sc.version)
# TODO create spark session with jdbc driver path
# print (df.select('*'))

21/11/30 07:46:26 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
21/11/30 07:46:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark-3.0.0-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-13b38014-b0c5-44e9-9ab7-3333835cd693;1.0
	confs: [default]
	found com.amazon.deequ#deequ;1.2.2-spark-3.0 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found org.scala-lang#scala-reflect;2.12.1 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransform

3.0.0


In [4]:
server_url = "127.0.0.1" # think what should be here
database_name = "AdventureWorks2019"
url = server_url + ";" + "databaseName=" + database_name + ";"

table = "Production.Product"
user = "***" # your user creds here
password  = "***" # your user creds here

jdbcDF = spark.read.format("jdbc") \
    .option("driver" , "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("url", f"jdbc:sqlserver://127.0.0.1:1433;databaseName={database_name}") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .load()

jdbcDF.show()
# df = spark.createDataFrame(jdbcDF)
# df.printSchema()
# TODO connect to DB with Spark using JDBC connection to read the data

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-

In [17]:
### Data Analyzers section
# TODO analyze data here
jdbcDF.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- ProductNumber: string (nullable = true)
 |-- MakeFlag: boolean (nullable = true)
 |-- FinishedGoodsFlag: boolean (nullable = true)
 |-- Color: string (nullable = true)
 |-- SafetyStockLevel: short (nullable = true)
 |-- ReorderPoint: short (nullable = true)
 |-- StandardCost: decimal(19,4) (nullable = true)
 |-- ListPrice: decimal(19,4) (nullable = true)
 |-- Size: string (nullable = true)
 |-- SizeUnitMeasureCode: string (nullable = true)
 |-- WeightUnitMeasureCode: string (nullable = true)
 |-- Weight: decimal(8,2) (nullable = true)
 |-- DaysToManufacture: integer (nullable = true)
 |-- ProductLine: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- ProductSubcategoryID: integer (nullable = true)
 |-- ProductModelID: integer (nullable = true)
 |-- SellStartDate: timestamp (nullable = true)
 |-- SellEndDate: timestamp (nullable = true)
 |-- Disco

In [25]:
list_columns = jdbcDF.columns
print(list_columns)


['ProductID', 'Name', 'ProductNumber', 'MakeFlag', 'FinishedGoodsFlag', 'Color', 'SafetyStockLevel', 'ReorderPoint', 'StandardCost', 'ListPrice', 'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode', 'Weight', 'DaysToManufacture', 'ProductLine', 'Class', 'Style', 'ProductSubcategoryID', 'ProductModelID', 'SellStartDate', 'SellEndDate', 'DiscontinuedDate', 'rowguid', 'ModifiedDate']


In [15]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(jdbcDF) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(CountDistinct("ProductID")) \
                    .addAnalyzer(Distinctness("ProductID")) \
                    .addAnalyzer(Maximum("ProductID")) \
                    .addAnalyzer(Minimum("ProductID")) \
                    .addAnalyzer(Mean("ProductID")) \
                    .addAnalyzer(Completeness("ProductID")) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()



+-----------+--------------------+-------------+-----------------+
|     entity|            instance|         name|            value|
+-----------+--------------------+-------------+-----------------+
|Mutlicolumn|ProductID,Name,Pr...|CountDistinct|            504.0|
|Mutlicolumn|ProductID,Name,Pr...| Distinctness|              1.0|
|    Dataset|                   *|         Size|            504.0|
|     Column|           ProductID|      Maximum|            999.0|
|     Column|           ProductID|      Minimum|              1.0|
|     Column|           ProductID|         Mean|673.0396825396825|
|     Column|           ProductID| Completeness|              1.0|
+-----------+--------------------+-------------+-----------------+



In [36]:
### Data profiling section
# TODO profile data here
from pydeequ.profiles import *
result = ColumnProfilerRunner(spark) \
        .onData(jdbcDF) \
        .run()
for col, profile in result.profiles.items():
    print(profile)


StandardProfiles for column: ModifiedDate: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 2,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": null
}
NumericProfiles for column: ReorderPoint: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 6,
    "dataType": "Integral",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": [
        [
            "45",
            5,
            0.00992063492063492
        ],
        [
            "750",
            156,
            0.30952380952380953
        ],
        [
            "375",
            167,
            0.33134920634920634
        ],
        [
            "75",
            97,
            0.19246031746031747
        ],
        [
            "600",
            25,
            0.0496031746031746
        ],
        [
            "3",
            54,
            0.10714285714285714
        ]
    ],
    "kll": "None",
    "mean": 401.36309523809

In [40]:
### Constraint Suggestions section
# TODO find meaninful constraints here
from pydeequ.suggestions import *
import json

suggestionResult = ConstraintSuggestionRunner(spark) \
                    .onData(jdbcDF) \
                    .addConstraintRule(DEFAULT()) \
                    .run()
# Constraint Suggestions in JSON format
print(json.dumps(suggestionResult, sort_keys=True, indent=10))


{
          "constraint_suggestions": [
                    {
                              "code_for_constraint": ".isComplete(\"ModifiedDate\")",
                              "column_name": "ModifiedDate",
                              "constraint_name": "CompletenessConstraint(Completeness(ModifiedDate,None))",
                              "current_value": "Completeness: 1.0",
                              "description": "'ModifiedDate' is not null",
                              "rule_description": "If a column is complete in the sample, we suggest a NOT NULL constraint",
                              "suggesting_rule": "CompleteIfCompleteRule()"
                    },
                    {
                              "code_for_constraint": ".isContainedIn(\"ReorderPoint\", [\"375\", \"750\", \"75\", \"3\", \"600\", \"45\"])",
                              "column_name": "ReorderPoint",
                              "constraint_name": "ComplianceConstraint(Compliance('ReorderPo

In [56]:
### Constraint Verification section
# TODO check selected constraints here and make beautify the report

from pydeequ.checks import *
from pydeequ.verification import *
check = Check(spark, CheckLevel.Warning, "Review Check")
checkResult = VerificationSuite(spark) \
                .onData(jdbcDF) \
                .addCheck(
                            check.hasSize(lambda x: x >= 3) \
                            .hasMin("ReorderPoint", lambda x: x == 3) \
                            .isComplete("ProductNumber") \
                            .isUnique("Name") \
                            .isContainedIn("MakeFlag", ["0", "1"]) \
                            .isNonNegative("DaysToManufacture")) \
                .run()
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()



  # check.hasMin('ReorderPoint', lambda x: x == 0) \
  #                           .isComplete('ProductNumber') \
  #                           .isUnique('Name') \
  #                           .isContainedIn('MakeFlag', [0, 1]) \
  #                           .isNonNegative('DaysToManufacture') \
  #                        ) \

+------------+-----------+------------+--------------------+-----------------+--------------------+
|       check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+------------+-----------+------------+--------------------+-----------------+--------------------+
|Review Check|    Warning|     Warning|SizeConstraint(Si...|          Failure|org.apache.spark....|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|org.apache.spark....|
|Review Check|    Warning|     Warning|CompletenessConst...|          Failure|org.apache.spark....|
|Review Check|    Warning|     Warning|UniquenessConstra...|          Success|                    |
|Review Check|    Warning|     Warning|ComplianceConstra...|          Failure|org.apache.spark....|
|Review Check|    Warning|     Warning|ComplianceConstra...|          Failure|org.apache.spark....|
+------------+-----------+------------+--------------------+-----------------+--------------------+


In [10]:
spark.sparkContext._gateway.shutdown_callback_server()
spark.stop()